## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

import os, sys
currentdir = os.path.dirname(os.path.realpath("Vacation_Search"))
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)


# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Ribeira Grande,PT,38.5167,-28.7000,66.60,88,40,17.27,scattered clouds
1,1,Yellowknife,CA,62.4560,-114.3525,-1.59,78,75,14.97,broken clouds
2,2,Busselton,AU,-33.6500,115.3333,54.05,65,99,8.95,overcast clouds
3,3,Rikitea,PF,-23.1203,-134.9692,74.82,69,1,10.51,clear sky
4,4,Albany,US,42.6001,-73.9662,40.23,50,71,1.99,broken clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
3,3,Rikitea,PF,-23.1203,-134.9692,74.82,69,1,10.51,clear sky
5,5,Padang,ID,-0.9492,100.3543,76.73,80,100,3.20,overcast clouds
7,7,Touros,BR,-5.1989,-35.4608,83.30,74,88,15.50,overcast clouds
8,8,Mana,GF,5.6592,-53.7767,82.51,74,51,9.60,broken clouds
11,11,Sao Felix Do Xingu,BR,-6.6447,-51.9950,81.75,77,100,1.79,overcast clouds
14,14,Cabo San Lucas,MX,22.8909,-109.9124,85.51,62,20,3.00,few clouds
16,16,Nampula,MZ,-15.1165,39.2666,78.75,69,0,8.05,clear sky
18,18,Atuona,PF,-9.8000,-139.0333,77.31,80,22,13.33,light rain
25,25,Lobito,AO,-12.3481,13.5456,76.55,83,78,6.76,broken clouds
28,28,Saint-Philippe,RE,-21.3585,55.7679,74.79,74,39,7.76,scattered clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.notnull().sum()

City_ID        240
City           240
Country        240
Lat            240
Lng            240
Max Temp       240
Humidity       240
Cloudiness     240
Wind Speed     240
Description    240
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
3,Rikitea,PF,74.82,clear sky,-23.1203,-134.9692,
5,Padang,ID,76.73,overcast clouds,-0.9492,100.3543,
7,Touros,BR,83.30,overcast clouds,-5.1989,-35.4608,
8,Mana,GF,82.51,broken clouds,5.6592,-53.7767,
11,Sao Felix Do Xingu,BR,81.75,overcast clouds,-6.6447,-51.9950,
14,Cabo San Lucas,MX,85.51,few clouds,22.8909,-109.9124,
16,Nampula,MZ,78.75,clear sky,-15.1165,39.2666,
18,Atuona,PF,77.31,light rain,-9.8000,-139.0333,
25,Lobito,AO,76.55,broken clouds,-12.3481,13.5456,
28,Saint-Philippe,RE,74.79,scattered clouds,-21.3585,55.7679,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"    
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
   
    hotels = requests.get(base_url, params=params).json() 
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"]!=""]
clean_hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
3,Rikitea,PF,74.82,clear sky,-23.1203,-134.9692,People ThankYou
5,Padang,ID,76.73,overcast clouds,-0.9492,100.3543,Mercure Padang
7,Touros,BR,83.30,overcast clouds,-5.1989,-35.4608,INN NEW HORIZON
8,Mana,GF,82.51,broken clouds,5.6592,-53.7767,LE SAMANA
11,Sao Felix Do Xingu,BR,81.75,overcast clouds,-6.6447,-51.9950,Rio Xingu Camping
14,Cabo San Lucas,MX,85.51,few clouds,22.8909,-109.9124,Hotel Tesoro Los Cabos
16,Nampula,MZ,78.75,clear sky,-15.1165,39.2666,BUTIK Hotel
18,Atuona,PF,77.31,light rain,-9.8000,-139.0333,Villa Enata
25,Lobito,AO,76.55,broken clouds,-12.3481,13.5456,Casa Rosa Hotel Residence
28,Saint-Philippe,RE,74.79,scattered clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))